# **Trabalho Prático 1: Atividade em conjunto de classificação de texto**
### Grupo: Bianca Muniz, Renan Cavalcante, Sara Magalhães e Talita Burbulhan
### Tema: Masterchef Brasil
**Objetivo**: Efetuar a classificação do corpus capturado na primeira aula e classes determinadas pelos grupos

**Passos**
* Criar corpus de treino
* Rodar algoritmos de treino e classificação
* Enviar 1 link para notebook no google colab por grupo

In [1]:
# Instalando e importando bibliotecas
!pip3 install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
# Criando uma lista para coletar os tweets com a query - biblioteca snscrape
tweets = []
# Pegando mil tweets em português da hashtag #masterchefbrasil
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('#masterchefbrasil lang:pt -filter:replies').get_items()):
    if i>1000:
        break
    tweets.append([tweet.id, tweet.date, tweet.content, tweet.username])

masterchef = pd.DataFrame(tweets)
masterchef.to_csv("masterchef_treino.csv", sep='\t', encoding='utf-8', index=False)

Importamos o arquivo CSV para o Google Drive e classificamos os tweets manualmente em três grupos (neutro, positivo ou negativo), até cada grupo ter 210 tweets. A planilha pode ser vista no [link](https://github.com/biamuniz/mineracao_masterchefbrasil/blob/main/tweets_treino.csv).

In [23]:
#Abrindo o arquivo CSV com os tweets classificados
dados = pd.read_csv(r'https://raw.githubusercontent.com/biamuniz/mineracao_masterchefbrasil/main/dados/tweets_treino.csv', sep=',')

In [6]:
dados

,id,data,tweet,user,classificação
0,1456834542627602433,2021-11-06 04:03:16+00:00,O que aconteceu com o Fogaça? Vendo agora o ep...,sferreiravic,Neutro
1,1456765784529768458,2021-11-05 23:30:03+00:00,"Em 2020, Marília participou do episódio especi...",RevistaISTOE,NaN
2,1456764222977761284,2021-11-05 23:23:50+00:00,Maria Rita e João Carlos Martins são os convid...,tvabordo,NaN
3,1456442462743015452,2021-11-05 02:05:17+00:00,Eu não entendo a perseguição com o Heitor e o ...,sabilisacrf,Positivo
4,1456316523019128840,2021-11-04 17:44:50+00:00,O horário de trabalho de todo mundo que trabal...,chefanahirata,NaN
...,...,...,...,...,...
996,1418040353115066378,2021-07-22 02:49:00+00:00,Gente…faço nem ideia de quem seja essa duda be...,nnalourenco,NaN
997,1418040340305620999,2021-07-22 02:48:57+00:00,Eu comecei não gostando da Amanda e agora ela ...,ccolemanholland,NaN
998,1418040289365827592,2021-07-22 02:48:45+00:00,Alguém sabe quem é Duda Beat?\n#masterchefbrasil,LuizMelloc,NaN
999,1418040242339209218,2021-07-22 02:48:34+00:00,Duda Beat no #masterchefbrasil é tudo pra mim 🥰🥰,lholhohelo,NaN


### **Filtrando os tweets classificados e salvando cada grupo em um arquivo de texto diferente**

Tweets negativos

In [8]:
negativo = dados.loc[dados['classificação'] == 'Negativo'] ## filtrando os tweets com classificação "Negativo" e armazenando no objeto "negativo"
tweet_negativo = negativo.drop(columns=['id', "data", "user", "classificação"]) ##excluindo colunas e deixando só a coluna de interesse (tweet)
tweet_negativo.to_csv("tweets_negativo.txt", sep='\t', encoding='utf-8', index = False) ## salvando o objeto em txt com o nome tweets_negativo.txt

Tweets positivos

In [9]:
positivo = dados.loc[dados['classificação'] == 'Positivo'] ## filtrando os tweets com classificação "Positivo" e armazenando no objeto "negativo"
tweet_positivo = positivo.drop(columns=['id', "data", "user", "classificação"]) ##excluindo colunas e deixando só a coluna de interesse (tweet)
tweet_positivo.to_csv("tweets_positivo.txt", sep='\t', encoding='utf-8', index = False) ## salvando o objeto em txt com o nome tweets_negativo.txt

Tweets neutros

In [10]:
neutro = dados.loc[dados['classificação'] == 'Neutro'] ## filtrando os tweets com classificação "Negativo" e armazenando no objeto "negativo"
tweet_neutro = neutro.drop(columns=['id', "data", "user", "classificação"]) ##excluindo colunas e deixando só a coluna de interesse (tweet)
tweet_neutro.to_csv("tweets_neutro.txt", sep='\t', encoding='utf-8', index = False) ## salvando o objeto em txt com o nome tweets_negativo.txt

### **Rodando o modelo**

In [11]:
!pip install nltk

In [16]:
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn.svm import LinearSVC

def divide(dados):    
    dados_new = []
    for palavra in dados:
        palavra_filter = [i.lower() for i in palavra.split()]
        dados_new.append(palavra_filter)
    return dados_new

def bag_of_words(palavras):
    return dict([(palavra, palavras.count(palavra)) for palavra in palavras])
    

def treina_classificadores(neutro,positivo,negativo): 
    posdados = []
    with open(positivo, 'r') as myfile:   
        reader = csv.reader(myfile, delimiter=',') 
        for val in reader:
            if len(val) > 0:
                posdados.append(val[0]) 

    negdados = [] 
    with open(negativo, 'r') as myfile:    
        reader = csv.reader(myfile, delimiter=',')
        for val in reader:
            if len(val) > 0:
                negdados.append(val[0])    

    neudados = [] 
    with open(neutro, 'r') as myfile:    
        reader = csv.reader(myfile, delimiter=',')
        for val in reader:
            if len(val) > 0:
                neudados.append(val[0])                  

    negfeats = [(bag_of_words(f), 'neg') for f in divide(negdados)] #pra cada tweet eu coloquei o marcado neg
    posfeats = [(bag_of_words(f), 'pos') for f in divide(posdados)] #pra cada tweet eu coloquei o marcado pos
    neufeats = [(bag_of_words(f), 'neu') for f in divide(neudados)] #pra cada tweet eu coloquei o marcado neu
   
    treino = negfeats + posfeats + neufeats

    
    classificadorME = MaxentClassifier.train(treino, 'GIS', trace=0, encoding=None, labels=None, gaussian_prior_sigma=0, max_iter = 1)
    classificadorSVM = SklearnClassifier(LinearSVC(), sparse=False)
    classificadorSVM.train(treino)
    classificadorNB = NaiveBayesClassifier.train(treino)
    
    return ([classificadorME,classificadorSVM,classificadorNB])


#aqui eu vou chamar a classificação de tweets novos        
#eu recebo uma lista de tweets e os objetos dos classificadores que eu treinei anteriormento
def classifica(sentencas, classificadores):
    ret = []                        
    for s in sentencas: #pra cada um dos tweets
        c = divide([s])
        feats= bag_of_words(c[0])
        classificacao = []
        classificacao.append(classificadores[1].classify(feats))
        classificacao.append(classificadores[2].classify(feats))
        classificacao.append(classificadores[0].classify(feats))
        ret.append(classificacao)
    return ret


In [22]:
######## MAIN             
neutro='tweets_neutro.txt'
positivo='tweets_positivo.txt'
negativo='tweets_negativo.txt'


classificadores = treina_classificadores(neutro,positivo,negativo)


sentences = ['Eu odeio o Sergio', 'espero que a Debi seja eliminada', 'os pratos estão com uma cara ótima!']

print (classifica(sentences, classificadores))


  



[['neg', 'pos', 'pos'], ['pos', 'pos', 'pos'], ['neu', 'neg', 'neg']]
